초기 소켓 버벅임 해결완료

In [ ]:
import pyaudio
import numpy as np
import math
import matplotlib.pyplot as plt
import socket
import time


MinDetect = 300
chunk = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

p = pyaudio.PyAudio()
plt.axis([0, 100, 0, 10000])
plt.ion()
plt.ylabel('Frequency and Amplitude')
plt.xlabel("time")
plt.show()
ffs = [0]
amp = [0]
line, = plt.plot(ffs)
line2, = plt.plot(amp)  # plot amp
babyDidCry = False
endFrame = 0
framesForWav = []

def do_some_stuffs_with_input(input_string):
    if input_string == "send":
        input_string = "..."
    else:
        input_string = input_string + "error"
    return input_string

def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan - 1, indnan + 1))), invert=True)]
    # first and last values of  cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size - 1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind] - x[ind - 1], x[ind] - x[ind + 1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                       & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0
    return ind
'''        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)'''



def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

def Pitch(signal):
    signal = np.fromstring(signal, 'Int16')
    maXabs = sum(abs(i) for i in signal) / float(len(signal))
    if maXabs > MinDetect:
        crossing = [math.copysign(1.0, s) for s in signal]
        index = find(np.diff(crossing));
        f0 = round(len(index) * RATE / (2 * np.prod(len(signal))))
        return f0, maXabs;
    else:
        return 0, 0

def detecting():
    global babyDidCry
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    output=True,
                    frames_per_buffer=chunk)
    HOST = ""
    PORT = 8091
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print ('Socket created')
    s.bind((HOST, PORT))
    print ('Socket bind complete')
    s.listen(1)
    print ('Socket now listening')
    conn, addr = s.accept()


    i=0
    while True:
        i=i+1
        data = stream.read(chunk, exception_on_overflow=False)
        Frequency, amplitude = Pitch(data)
        ffs.append(Frequency)
        amp.append(amplitude)
        line.set_xdata(np.arange(len(ffs)))
        line.set_ydata(ffs)
        line2.set_xdata(np.arange(len(amp)))
        line2.set_ydata(amp)
        #plt.draw()
        #plt.pause(0.001)
        if i < 100:
            plt.axis([0, 100, 0, 10000])
        else:
            plt.axis([0, i, 0, 10000])

        if (i + 1) % 100 == 0:
            avgPeak = []
            if np.count_nonzero(ffs[i - 99:]) != 0:
                peaks = detect_peaks(ffs[i - 99:], mph=2000, mpd=3)
                averageAmp = np.average(amp[i - 99:])
                if len(peaks) > 0:
                    for j in peaks:
                        avgPeak.append(ffs[i - 99])
                    avgPeakResult = np.average(avgPeak)
                    stdPeak = np.std(avgPeak)
                else:
                    avgPeakResult = 0
                    stdPeak = 0
                print(avgPeakResult, stdPeak, averageAmp, len(peaks))
                if np.average(avgPeak) <= 5000 and np.std(avgPeak) < 1000 and averageAmp > 20 and len(
                        peaks) >= 3 and len(peaks) < 14:
                    if babyDidCry == False:
                        babyDidCry = True
                        #endFrame = i + 500
                    print ("baby crying")
                    khSocket(conn, addr)
                    conn, addr = s.accept()
                    time.sleep(1)
                else:
                    print ("noise_1")
            else:
                print ("noise_2")



       # print('end i:',i)



       # conn.close()
    print("good")
    '''while True:  # ESC
        plt.pause(0.05)
'''

def khSocket(conn, addr):
    datak = conn.recv(1024)
    datak = datak.decode("utf8").strip()
    res = do_some_stuffs_with_input(datak)
    command = ("baby crying")
    conn.sendall(command.encode("utf-8"))
    conn.close()


detecting()




파이오디오를 이용한 음악파일 재생코드

In [ ]:
import pyaudio
import wave
import sys

class AudioFile:
    chunk = 1024

    def __init__(self, file):
     """ Init audio stream """
     self.wf = wave.open(file, 'rb')
     self.p = pyaudio.PyAudio()
     self.stream = self.p.open(
      format = self.p.get_format_from_width(self.wf.getsampwidth()),
      channels = self.wf.getnchannels(),
      rate = self.wf.getframerate(),
      output = True )

    def play(self): 
     """ Play entire file """ 
     data = self.wf.readframes(self.chunk) 
     while data != '': 
      self.stream.write(data) 
      data = self.wf.readframes(self.chunk) 

    def close(self): 
     """ Graceful shutdown """ 
     self.stream.close() 
     self.p.terminate() 

# Usage example for pyaudio 
a = AudioFile("jajangga.wav")
a.play() 
a.close()

자장가 추가된 라즈베리파이 전체코드 5/8

In [ ]:
import pyaudio
import numpy as np
import math
import matplotlib.pyplot as plt
import socket
import time
import pyaudio
import wave
import sys

class AudioFile:
    chunk = 1024

    def __init__(self, file):
     """ Init audio stream """
     self.wf = wave.open(file, 'rb')
     self.p = pyaudio.PyAudio()
     self.stream = self.p.open(
      format = self.p.get_format_from_width(self.wf.getsampwidth()),
      channels = self.wf.getnchannels(),
      rate = self.wf.getframerate(),
      output = True )

    def play(self):
     """ Play entire file """
     data = self.wf.readframes(self.chunk)
     while data != '':
      self.stream.write(data)
      data = self.wf.readframes(self.chunk)

    def close(self):
     """ Graceful shutdown """
     self.stream.close()
     self.p.terminate()




MinDetect = 300
chunk = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

p = pyaudio.PyAudio()
plt.axis([0, 100, 0, 10000])
plt.ion()
plt.ylabel('Frequency and Amplitude')
plt.xlabel("time")
plt.show()
ffs = [0]
amp = [0]
line, = plt.plot(ffs)
line2, = plt.plot(amp)  # plot amp
babyDidCry = False
endFrame = 0
framesForWav = []

def do_some_stuffs_with_input(input_string):
    if input_string == "send":
        input_string = "..."
    else:
        input_string = input_string + "error"
    return input_string

def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan - 1, indnan + 1))), invert=True)]
    # first and last values of  cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size - 1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind] - x[ind - 1], x[ind] - x[ind + 1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                       & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0
    return ind
'''        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)'''



def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

def Pitch(signal):
    signal = np.fromstring(signal, 'Int16')
    maXabs = sum(abs(i) for i in signal) / float(len(signal))
    if maXabs > MinDetect:
        crossing = [math.copysign(1.0, s) for s in signal]
        index = find(np.diff(crossing));
        f0 = round(len(index) * RATE / (2 * np.prod(len(signal))))
        return f0, maXabs;
    else:
        return 0, 0

def detecting():

    global babyDidCry
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    output=True,
                    frames_per_buffer=chunk)
    HOST = ""
    PORT = 8091
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print ('Socket created')
    s.bind((HOST, PORT))
    print ('Socket bind complete')
    s.listen(1)
    print ('Socket now listening')
    conn, addr = s.accept()


    i=0
    while True:
        i=i+1
        data = stream.read(chunk, exception_on_overflow=False)
        Frequency, amplitude = Pitch(data)
        ffs.append(Frequency)
        amp.append(amplitude)
        line.set_xdata(np.arange(len(ffs)))
        line.set_ydata(ffs)
        line2.set_xdata(np.arange(len(amp)))
        line2.set_ydata(amp)
        #plt.draw()
        #plt.pause(0.001)
        if i < 100:
            plt.axis([0, 100, 0, 10000])
        else:
            plt.axis([0, i, 0, 10000])

        if (i + 1) % 100 == 0:
            avgPeak = []
            if np.count_nonzero(ffs[i - 99:]) != 0:
                peaks = detect_peaks(ffs[i - 99:], mph=2000, mpd=3)
                averageAmp = np.average(amp[i - 99:])
                if len(peaks) > 0:
                    for j in peaks:
                        avgPeak.append(ffs[i - 99])
                    avgPeakResult = np.average(avgPeak)
                    stdPeak = np.std(avgPeak)
                else:
                    avgPeakResult = 0
                    stdPeak = 0
                print(avgPeakResult, stdPeak, averageAmp, len(peaks))
                if np.average(avgPeak) <= 5000 and np.std(avgPeak) < 1000 and averageAmp > 20 and len(
                        peaks) >= 3 and len(peaks) < 14:
                    if babyDidCry == False:
                        babyDidCry = True
                        #endFrame = i + 500
                    print ("baby crying")
                    khSocket(conn, addr)
                    conn, addr = s.accept()
                    jajangga = AudioFile("jajangga.wav")
                    jajangga.play()
                    jajangga.close()
                    time.sleep(1)
                    stream = p.open(format=FORMAT,
                                    channels=CHANNELS,
                                    rate=RATE,
                                    input=True,
                                    output=True,
                                    frames_per_buffer=chunk)
                    data = stream.read(chunk, exception_on_overflow=False)
                    Frequency, amplitude = Pitch(data)
                    ffs.append(Frequency)
                    amp.append(amplitude)
                else:
                    print ("noise_1")
            else:
                print ("noise_2")



       # print('end i:',i)



       # conn.close()
    print("good")
    '''while True:  # ESC
        plt.pause(0.05)
'''

def khSocket(conn, addr):
    datak = conn.recv(1024)
    datak = datak.decode("utf8").strip()
    res = do_some_stuffs_with_input(datak)
    command = ("baby crying")
    conn.sendall(command.encode("utf-8"))
    conn.close()



detecting()


